<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Модель-до-шифрования" data-toc-modified-id="Модель-до-шифрования-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Модель до шифрования</a></span></li><li><span><a href="#Шифрование" data-toc-modified-id="Шифрование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Шифрование</a></span></li><li><span><a href="#Модель-на-зашифрованных-данных" data-toc-modified-id="Модель-на-зашифрованных-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Модель на зашифрованных данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Алгорим-шифрования" data-toc-modified-id="Алгорим-шифрования-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Алгорим шифрования</a></span><ul class="toc-item"><li><span><a href="#Обобснование" data-toc-modified-id="Обобснование-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Обобснование</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

<div class="alert alert-info">
При загрузке нового файла тут всё сбилось, я вернул на места, но это не точно</div>

# Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.  
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.  

# План проекта

Загрузим и изучим данные.  
Ответим на вопрос и обоснуем решение.  
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)  
a. Изменится. Приведем примеры матриц.  
b. Не изменится. Укажим, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.  
Предложим алгоритм преобразования данных для решения задачи. Обоснуем, почему качество линейной регрессии не поменяется.  
Запрограммируем этот алгоритм, применив матричные операции. Проверим, что качество линейной регрессии из sklearn не   отличается до и после преобразования. Применим метрику R2.  

# Описание данных

**Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.  
**Целевой признак**: количество страховых выплат клиенту за последние 5 лет.  

In [3]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from random import randint

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [4]:
try:
    data = pd.read_csv('insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')

## Обзор данных

In [5]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [7]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Пропусков нет

In [8]:
data.duplicated().sum()

153

Думаю, учитывая наши данные, такое кол-во одинаковых значений имеет место быть

### Вывод

Данные впорядке, проводим эксперименты дальше

## Модель до шифрования

In [9]:
X = data.drop(['Страховые выплаты'], axis=1)
y = data['Страховые выплаты']

In [10]:
lr = LinearRegression()
lr.fit(X,y)
lr_pred = lr.predict(X)
score_above = round(r2_score(y, lr_pred), 4)
score_above

0.4249

## Шифрование

Создадим рандомную квадратную, обратимую матрицу для шифрования наших данных с кол-вом строк равным кол-ву целевых признаков

In [11]:
rand = np.random.RandomState(42)
matrix = rand.randint(10, size=(X.shape[1], X.shape[1]))  
matrix

array([[6, 3, 7, 4],
       [6, 9, 2, 6],
       [7, 4, 3, 7],
       [7, 2, 5, 4]])

In [12]:
# Проверим матрицу на обратимость
inv = inv(matrix)

Матрица обратима, продолжаем.

Проведем скалярное произведение данных клиентов и нашего ключа

In [13]:
X_key = X @ matrix
X_key.columns = X.columns

In [14]:
X_key

,Пол,Возраст,Зарплата,Члены семьи
0,347459.0,198774.0,148894.0,347454.0
1,266283.0,152416.0,114097.0,266280.0
2,147174.0,84261.0,63058.0,147174.0
3,292040.0,166993.0,125152.0,292034.0
4,182874.0,104655.0,78363.0,182872.0
...,...,...,...,...
4995,250082.0,143056.0,107166.0,250076.0
4996,367011.0,209908.0,157273.0,367008.0
4997,237434.0,135784.0,101750.0,237428.0
4998,229059.0,131007.0,98166.0,229048.0


## Модель на зашифрованных данных

In [15]:
lr = LinearRegression()
lr.fit(X_key,y)
lr_pred = lr.predict(X_key)
score_below = round(r2_score(y, lr_pred), 4)

In [16]:
score_below == score_above

True

### Вывод

Качество модели не изменилось после шифрования

## Алгорим шифрования

Составление рандомной матрицы - ключа.  
Проверка матрицы на обратимость.  
Получение матрицы преобразованных признаков путем умножения признаков на ключ.  
Применение алгоритма на преобразованных признаках .  

### Обобснование

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$

**Доказательство:**


$
a =  X((X^T X)^{-1} X^T y) 
$

Домножим Х на матрицу P

$a_1 = XP((XP)^T(XP))^{-1}(XP)^Ty = XP(P^T (X^T X) P)^{-1}(XP)^Ty = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}(XP)^Ty = $

$ = XE(X^TX)^{-1}EX^Ty = X(X^TX)^{-1}X^Ty = a$

Как мы видим, итоговый расчет предсказаний не изменился при шифрованиее

## Вывод

Мы изучили данные клиентов, провели предобработку, и обучили модель линейной регрессии на исходных данных, получив в результате r2 = 0.4249.  
После чего разработали алгоритм шифрования, применили его и проверили модель на новых данных.  
В итоге качество модели не изменилось.